In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)


<h2 align="center">
  Código criados para localizar as cidades dos dados via latitude e longitude
</h2>


<p align="center">
  Executar somente para o arquivo csv baixado via Google Drive
</p>


<div style = "text-align: center;">
<strong>tempo de processamento longo por causa da API da biblioteca geopy</strong>
</div>


In [2]:
# Biblioteca usada para localizar o nome da cidade via latitude e longitude
from geopy.geocoders import Nominatim
geolocal = Nominatim(user_agent="tpDados")


<p align="center">
  Convertendo o tipo da coluna cidade para objeto por questões de compatibilidade com o geopy
  
</p>

In [ ]:
df["CIDADE"] = df["CIDADE"].astype("object")

In [ ]:
for indice, linha in df.iterrows():
    if pd.isna(linha["LATITUDE"]) and pd.isna(linha["LONGITUDE"]):
        continue
    if pd.isna(linha["CIDADE"]) and pd.notna(linha["LATITUDE"]) and pd.notna(linha["LONGITUDE"]):
        try:
            lat = linha["LATITUDE"]  # Pega a latitude da linha atual
            long = linha["LONGITUDE"]  # Pega a longitude da linha atual
            local = geolocal.reverse(f"{lat}, {long}")
            
            # Tenta pegar "city", se não existir, tenta "town", se não existir, preenche com NaN
            df.at[indice, "CIDADE"] = local.raw["address"].get("city") or local.raw["address"].get("town", np.nan)
        except ValueError:
            erro_df.loc[len(erro_df)] = [indice, lat, long]
            df.at[indice, "CIDADE"] = np.nan



<p align="center">
  Comando para salvar o DataFrame em um arquivo csv
</p>

In [164]:
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')

<p align="center">
  
</p>

In [162]:
erro_df = pd.DataFrame(columns= ["Indice", "Latitude", "Longitude"] )
erro_df.dtypes


Indice       object
Latitude     object
Longitude    object
dtype: object

In [174]:
ultimo = pd.DataFrame(columns= ["Ultimo Indice"])

ultimo.loc[len(ultimo)] = indice

ultimo.loc[0]

ultimo.to_csv('dados/ultimoIndice.csv', index=False, sep=';', encoding='utf-8')


In [165]:

erro_df.to_csv('dados/erros.csv', index=False, sep=';', encoding='utf-8')